# Module 2
## Lab 2: Back to Libraries

Now that we understand HOW sharding is done, we can understand how this tool is utilized by  libaries to optimize your model inference/training across multiple accelerators. If you want a deeper dive on individual sharding patterns that make up parallism patterns, and the collectives associated with them, we highly recommend [this chapter](https://jax-ml.github.io/scaling-book/sharding/) of **How to Scale your Model**.

In this notebook though, we will focus on the different parallism strategies that can be implemented by these libaries, and what the considerations with them are.

### Parallism Patterns
This lab will be a hands on representation of common parallism patterns. If you want a achemic deep dive, we suggest you read through [The Ultra-Scale Playbook](https://huggingface.co/spaces/nanotron/ultrascale-playbook?section=high_level_overview).

> Note we will be using Deepspeed here but this could just as easily be done with any distributed inference/training framework, they have similar interfaces

We will be covering the following:
- Data Parallism
- Tensor Parallism (Like we covered last chapter)
- Pipeline Parallism
- ZeRO

There are other strategies as well, and likely more will emerge, but these are the main ones and others will follow similar concepts. 

Let's start by installing our libraries.


In [2]:
import os
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
os.chdir(parent_dir)

In [ ]:
%pip install transformers accelerate deepspeed

Now we will run through the different types of parallism. Keep an eye out for `OUTPUT BREAKDOWN` to see what the token generation and cost looks like. You may see some warnings/errors but these are benign and can be ignored.

#### Data Parallel
**How it works:**
Each GPU has a full copy of the model. Batches are split across GPUs. Gradients are synced after the backward pass.

✅ Advantages:
- Easy to implement (e.g., torch.nn.DataParallel, DDP)
- Scales well for small to mid-sized models
- No model code changes required
- Best for large data sets and small models

❌ Disadvantages:
- Inefficient for very large models (can't fit on one GPU)
- All-reduce on gradients becomes a bottleneck at high scale


Let's see it in action. We'll use a smaller model (1B) because we aren't splitting the model this time.

In [55]:
import importlib
import src.utils.model_utils as mutils
importlib.reload(mutils)

# Create a temporary deepspeed config file
ds_config = {
    "tensor_parallel": {
        "enabled": False
    },
    "enable_cuda_graph": False,
}

# Running with 1 GPU
results = mutils.run_distributed_benchmark(
    model_name="NousResearch/Llama-3.2-1B",
    seq_len=32,
    min_new_tokens=1,
    batch_sizes=[100],
    dtype=torch.bfloat16,
    sharding=False,
    world_size=1, # Number of GPUs
    ds_config=ds_config
)
mutils.reset_distributed_and_clear_memory()

# Copy the model to 2 GPUs and split the data
results = mutils.run_distributed_benchmark(
    model_name="NousResearch/Llama-3.2-1B",
    seq_len=32,
    min_new_tokens=1,
    batch_sizes=[100],
    dtype=torch.bfloat16,
    sharding=True,
    world_size=2, # Number of GPUs
    ds_config=ds_config
)
mutils.reset_distributed_and_clear_memory()

[2025-05-07 15:51:40,921] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cuda (auto detect)

🔁 Running batch size = 100
Batch=100 | Seq=32+1
Elapsed GPU time: 0.1788s | TFLOP/s: 45.6 | AI: 3264.83 FLOP/B
--------- OUTPUT BREAKDOWN ---------
🧠 Tokens generated: 100
⚡ Throughput: 559.22767 tokens/sec
⏱️ Total time: 0.17882 sec
💸 Cost per 1M tokens: $0.60103
------------------------------------
✅ Distributed env torn down and memory cleared.
[2025-05-07 15:51:48,979] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2025-05-07 15:51:49,000] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cuda (auto detect)

🔁 Running batch size = 100
[2025-05-07 15:51:51,221] [INFO] [logging.py:128:log_dist] [Rank -1] DeepSpeed info: version=0.16.2, git-hash=unknown, git-branch=unknown
[2025-05-07 15:51:51,221] [WARNING] [config_utils.py:70:_process_deprecated_field] Config parameter replace_method is depre

Great! As you can see distributed data can allow us to process our workload much faster, this is similar to how we previously used batching to improove price/performance. Here we can do something similar. But once our model is large enough this stragegy will no longer work, and we'll run into the same issues as before.

#### Tensor Parallism
**How it works:**
Individual layers are sharded across GPUs — e.g., split matrix rows/columns in linear layers.

✅ Advantages:
- Enables sharding of very large models/layers
- Reduces per-GPU memory usage
- Exploits fine-grained parallelism within layers

❌ Disadvantages:
- Requires deep model rewrites or tools like DeepSpeed/FSDP
- Requires custom communication (e.g., all_gather, reduce_scatter)
- Collective comms (e.g., NCCL) can dominate runtime if not optimized

In [48]:
import json
import importlib
import torch
import src.utils.model_utils as mutils
importlib.reload(mutils)

seq_len = 32
min_new_tokens = 1
world_size = 2
max_tokens =  seq_len + min_new_tokens

ds_config = {
    "replace_with_kernel_inject": True,
    "enable_cuda_graph": False,
    "tensor_parallel": {
        "enabled": True,
        "tp_size": world_size
    },
    # Optional tuning knobs to constrain token planning
    "max_tokens": max_tokens
}

results = mutils.run_distributed_benchmark(
    model_name="NousResearch/Nous-Hermes-Llama2-13b",
    seq_len=32,
    min_new_tokens=1,
    batch_sizes=[1],
    dtype=torch.bfloat16,
    sharding=True,
    world_size=2, # Number of GPUs
    ds_config=ds_config
)
mutils.reset_distributed_and_clear_memory()

[2025-05-07 15:21:48,942] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2025-05-07 15:21:48,950] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cuda (auto detect)

🔁 Running batch size = 1


Loading checkpoint shards: 100%|██████████| 3/3 [00:00<00:00,  7.48it/s]
/home/ec2-user/.local/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/ec2-user/.local/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed

[2025-05-07 15:21:51,329] [INFO] [logging.py:128:log_dist] [Rank -1] DeepSpeed info: version=0.16.2, git-hash=unknown, git-branch=unknown
[2025-05-07 15:21:51,329] [WARNING] [config_utils.py:70:_process_deprecated_field] Config parameter replace_method is deprecated. This parameter is no longer needed, please remove from your call to DeepSpeed-inference
[2025-05-07 15:21:51,329] [INFO] [logging.py:128:log_dist] [Rank -1] quantize_bits = 8 mlp_extra_grouping = False, quantize_groups = 1
[2025-05-07 15:21:51,332] [INFO] [comm.py:652:init_distributed] cdb=None
[2025-05-07 15:21:51,371] [INFO] [logging.py:128:log_dist] [Rank 0] DeepSpeed-Inference config: {'layer_id': 0, 'hidden_size': 5120, 'intermediate_size': 13824, 'heads': 40, 'num_hidden_layers': -1, 'dtype': torch.bfloat16, 'pre_layer_norm': True, 'norm_type': <NormType.RMSNorm: 3>, 'local_rank': -1, 'stochastic_mode': False, 'epsilon': 1e-05, 'mp_size': 2, 'scale_attention': True, 'triangular_masking': True, 'local_attention': Fals

Loading checkpoint shards: 100%|██████████| 3/3 [00:00<00:00,  7.31it/s]
/home/ec2-user/.local/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/ec2-user/.local/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed

ninja: no work to do.
Time to load transformer_inference op: 0.020978927612304688 seconds
ninja: no work to do.
Time to load transformer_inference op: 0.02348494529724121 seconds
------------------------------------------------------
Free memory : 7.072205 (GigaBytes)  
Total memory: 22.045044 (GigaBytes)  
Requested memory: 0.015776 (GigaBytes) 
Setting maximum total tokens (input + output) to 33 
WorkSpace: 0x7f10c1400000 
------------------------------------------------------
Batch=1 | Seq=32+1
Elapsed GPU time: 0.0664s | TFLOP/s: 6.6 | AI: 33.00 FLOP/B
🧠 Tokens generated: 2
⚡ Throughput: 30.05825 tokens/sec
⏱️ Total time: 0.06654 sec
💸 Cost per 1M tokens: $11.18199


Traceback (most recent call last):
  File "/usr/local/lib/python3.12/multiprocessing/util.py", line 303, in _run_finalizers
    finalizer()
  File "/usr/local/lib/python3.12/multiprocessing/util.py", line 227, in __call__
    res = self._callback(*self._args, **self._kwargs)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/multiprocessing/synchronize.py", line 87, in _cleanup
    sem_unlink(name)
FileNotFoundError: [Errno 2] No such file or directory
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/multiprocessing/util.py", line 303, in _run_finalizers
    finalizer()
  File "/usr/local/lib/python3.12/multiprocessing/util.py", line 227, in __call__
    res = self._callback(*self._args, **self._kwargs)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/multiprocessing/synchronize.py", line 87, in _cleanup
    sem_unlink(name)
FileNotFoundError: [Errno 2] No such file or directory


[
  {
    "batch_size": 1,
    "world_size": 2,
    "avg_time_seconds": 0.06635622406005859,
    "local_gflops": 6636.48785683498,
    "aggregated_gflops": 6636.48785683498,
    "total_flops": 440372275200,
    "estimated_memory_bytes": 13345280000,
    "arithmetic_intensity": 32.998354114713216,
    "cost_per_1m_tokens": 11.181992049225501
  }
]✅ Distributed env torn down and memory cleared.



This Tensor parallism is exactly the same as what we demonstrated in the last lab. This allows us to launch a much larger model by utilizing more GPUs. 

#### Pipeline Parallelism
**How it works:**
Each GPU holds a different stage of the model. Batches are split into micro-batches and passed between GPUs sequentially.

✅ Advantages:
Works well for extremely deep models
- Spreads compute and memory across GPUs
- Compatible with tensor parallelism for hybrid scaling

❌ Disadvantages:
- Latency due to pipeline bubbles (idle GPUs while others compute)
- Complex micro-batching & scheduling
- Harder to load balance if layers are uneven in cost

In [52]:
import json
import importlib
import torch
import src.utils.model_utils as mutils
importlib.reload(mutils)

seq_len = 32
min_new_tokens = 1
world_size = 2
max_tokens =  seq_len + min_new_tokens

ds_config = {
    "pipeline_parallel": {
        "enabled": True,
        "pp_size": world_size
    },
    "max_tokens": max_tokens
}

results = mutils.run_distributed_benchmark(
    model_name="NousResearch/Nous-Hermes-Llama2-13b",
    seq_len=32,
    min_new_tokens=1,
    batch_sizes=[1],
    dtype=torch.bfloat16,
    sharding="tensor",
    world_size=2, # Number of GPUs
    ds_config=ds_config
)
mutils.reset_distributed_and_clear_memory()

[2025-05-07 15:39:39,445] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2025-05-07 15:39:39,471] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cuda (auto detect)

🔁 Running batch size = 1


Loading checkpoint shards: 100%|██████████| 3/3 [00:00<00:00,  7.67it/s]
/home/ec2-user/.local/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/ec2-user/.local/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed

[2025-05-07 15:39:42,517] [INFO] [logging.py:128:log_dist] [Rank -1] DeepSpeed info: version=0.16.2, git-hash=unknown, git-branch=unknown
[2025-05-07 15:39:42,573] [INFO] [logging.py:128:log_dist] [Rank -1] DeepSpeed info: version=0.16.2, git-hash=unknown, git-branch=unknown


/home/ec2-user/.local/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/ec2-user/.local/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/ec2-user/.local/lib/python3.12/site-packages/tra

ProcessRaisedException: 

-- Process 1 terminated with the following error:
Traceback (most recent call last):
  File "/home/ec2-user/.local/lib/python3.12/site-packages/torch/multiprocessing/spawn.py", line 90, in _wrap
    fn(i, *args)
  File "/home/ec2-user/environment/src/utils/model_utils.py", line 403, in _distributed_worker
    res = benchmark_batch_sizes(
          ^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ec2-user/environment/src/utils/model_utils.py", line 57, in benchmark_batch_sizes
    elapsed_s, tokens_generated, metrics, cost = benchmark_llm(
                                                 ^^^^^^^^^^^^^^
  File "/home/ec2-user/environment/src/utils/model_utils.py", line 201, in benchmark_llm
    model = load_sharded_model(model_name, dtype, sharding, world_size, seq_len, min_new_tokens, ds_config)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ec2-user/environment/src/utils/model_utils.py", line 144, 